# Check the implementation from:
https://geemap.org/notebooks/31_unsupervised_classification/ 

In [1]:
import gc
gc.collect()

10

In [2]:
import geemap
import ee

In [3]:
Map = geemap.Map()
Map.add_basemap("SATELLITE")
# Map

## Change to Sentinel 2 TOA and check cloud covers; change ROI and clustering ROI for better accuracy!

In [4]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799]) # Michigan
# point = ee.Geometry.Point([18.08, 59.32]) # Stockholm
# region = ee.Geometry.BBox(17.82, 59.22, 18.34, 59.42) # Stockholm
region = ee.Geometry.BBox(17.70, 59.20, 18.70, 59.60) # Stockholm

image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(region)
    .filterDate('2023-06-01', '2023-08-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .mosaic()
    .unitScale(0, 10000)
    .clip(region) # Don't clip as we want to analyse the whole image
    # .select('B[1-7]')
)

# True Color Display:
vis_params = {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(region, 8)
Map.addLayer(image, vis_params, "Sentinel-2")

In [5]:
props = geemap.image_props(image)
props.getInfo()
# props.get('IMAGE_DATE').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties
# props.get('CLOUD_COVER').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties

{'NOMINAL_SCALE': 111319.49079327357,
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL',
  'TCI_R',
  'TCI_G',
  'TCI_B',
  'MSK_CLDPRB',
  'MSK_SNWPRB',
  'QA10',
  'QA20',
  'QA60']}

# Construct Training Data Set:

In [6]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')
input = image.addBands([ndvi, ndwi])

# Define the input features for the classifier
# inputFeatures = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'ndvi', 'ndwi']
inputFeatures = ['B2', 'B3', 'B4', 'B8', 'B11', 'ndvi', 'ndwi']

In [7]:
# Make the training dataset.
training = input.select(inputFeatures).sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000, # Default, given
        # 'numPixels': 10000,
        # 'seed': 0,
        # 'geometries': True,  # Set this to False to ignore geometries
    }
)

# Plot the training sample data (not really necessary)
# Map.addLayer(training, {}, 'training', False)
# Map

In [8]:
# Instantiate the clusterer and train it.
n_clusters = 15
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [9]:
# Cluster the input using the trained clusterer.
result = input.select(inputFeatures).cluster(clusterer)
geemap.image_props(result)

# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'clusters')
# Map.addLayer(result, {}, 'clusters')
# Map

In [10]:
# Define 15 colors to distinguish the clusters and look at what they are!
# legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
# legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
legend_keys = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']
legend_colors = ['#e6194B','#3cb44b','#ffe119','#4363d8','#f58231','#911eb4','#42d4f4','#f032e6','#bfef45','#fabed4','#469990','#9A6324','#800000','#808000','#000075']

Map.addLayer(
    result, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    keys=legend_keys, colors=legend_colors, position='bottomright'
)
Map

Map(center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position', 'transparent_…

Water: 1 (high shallow), 2 (deep), 8 (mid shallow), 13 (shallow), 14 (?, close to outer seas)
Vegetation: 0 (larger), 3, 4 (slight veg) , 5 (tall), 11 (more disperse veg)
Urban: 7 (darker/road), 9 (metallic rooftop or cloud)
Bare Field: 6 (a bit more veg), 10 (barer), 12 (even more barer)


Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

# Define 
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Remap the spectral classes from clusters:
fromClusters = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
toClasses = [2, 3, 0, 2, 2, 3, 1, 3, 0, 1, 3, 2, 2, 3, 1] # To change!

# Reclassify the map
result = result.remap(fromClusters, toClasses)

Map.addLayer(
    result, {'min': 0, 'max': 4, 'palette': legend_colors}, 'Labelled clusters 2'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

In [11]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

# Image Export (optional)
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

geemap.ee_export_image(result, filename=out_file, scale=90)
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)

geemap.show_youtube('N7rK2aV1R4c')

import geemap.foliumap as gf
# import geemap as gf
Map2 = gf.Map()
Map2.centerObject(region, 8)
Map2.add_basemap("SATELLITE")

tile_img = gf.ee_tile_layer(image, vis_params, 'Sentinel-2 True Colour')
tile_cluster = gf.ee_tile_layer(result, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Sentinel-2 Clustered Unclassified')


Map2.split_map(left_layer=tile_img, right_layer=tile_cluster)
Map2.addLayerControl

Map2.add_legend(
    labels=legend_keys, colors=legend_colors, position='bottomright'
)
# Map2.add_legend(
#     keys=legend_keys, colors=legend_colors, position='bottomright'
# )

Map2